# 準備

## GoogleDriveへ接続

In [1]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## パッケージインストール

In [ ]:
!pip install azure-ai-formrecognizer==3.3.0

# PaddleOCR

## テスト

In [27]:
# import libraries
import os
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

# set `<your-endpoint>` and `<your-key>` variables with the values from the Azure portal
endpoint = "https://form-recognizer-2023.cognitiveservices.azure.com/"
key = "c62ee706ee774c2bac2d8a6917583051"

def format_polygon(polygon):
    if not polygon:
        return "N/A"
    return ", ".join(["[{}, {}]".format(p.x, p.y) for p in polygon])

def analyze_layout():
    # sample document
    formUrl = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-layout.pdf"

    document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key)
    )

    poller = document_analysis_client.begin_analyze_document_from_url(
            "prebuilt-layout", formUrl)
    result = poller.result()

    response = []

    for page in result.pages:

        contents = []

        # print("----Analyzing layout from page #{}----".format(page.page_number))
        # print(
        #     "Page has width: {} and height: {}, measured with unit: {}".format(
        #         page.width, page.height, page.unit
        #     )
        # )
        contents.append("----Analyzing layout from page #{}----".format(page.page_number))
        contents.append(
            "Page has width: {} and height: {}, measured with unit: {}".format(
                page.width, page.height, page.unit
            )
        )

        for line_idx, line in enumerate(page.lines):
            words = line.get_words()
            # print(
            #     "...Line # {} has word count {} and text '{}' within bounding box '{}'".format(
            #         line_idx,
            #         len(words),
            #         line.content,
            #         format_polygon(line.polygon),
            #     )
            # )
            contents.append(
                "...Line # {} has word count {} and text '{}' within bounding box '{}'".format(
                    line_idx,
                    len(words),
                    line.content,
                    format_polygon(line.polygon),
                )
            )

            for word in words:
                # print(
                #     "......Word '{}' has a confidence of {}".format(
                #         word.content, word.confidence
                #     )
                # )
                contents.append(
                    "......Word '{}' has a confidence of {}".format(
                        word.content, word.confidence
                    )
                )

        for selection_mark in page.selection_marks:
            # print(
            #     "...Selection mark is '{}' within bounding box '{}' and has a confidence of {}".format(
            #         selection_mark.state,
            #         format_polygon(selection_mark.polygon),
            #         selection_mark.confidence,
            #     )
            # )
            contents.append(
                "...Selection mark is '{}' within bounding box '{}' and has a confidence of {}".format(
                    selection_mark.state,
                    format_polygon(selection_mark.polygon),
                    selection_mark.confidence,
                )
            )

        for table_idx, table in enumerate(result.tables):
            # print(
            #     "Table # {} has {} rows and {} columns".format(
            #         table_idx, table.row_count, table.column_count
            #     )
            # )
            contents.append(
                "Table # {} has {} rows and {} columns".format(
                    table_idx, table.row_count, table.column_count
                )
            )
            for region in table.bounding_regions:
                # print(
                #     "Table # {} location on page: {} is {}".format(
                #         table_idx,
                #         region.page_number,
                #         format_polygon(region.polygon),
                #     )
                # )
                contents.append(
                    "Table # {} location on page: {} is {}".format(
                        table_idx,
                        region.page_number,
                        format_polygon(region.polygon),
                    )
                )
            for cell in table.cells:
                # print(
                #     "...Cell[{}][{}] has content '{}'".format(
                #         cell.row_index,
                #         cell.column_index,
                #         cell.content,
                #     )
                # )
                contents.append(
                    "...Cell[{}][{}] has content '{}'".format(
                        cell.row_index,
                        cell.column_index,
                       cell.content,
                    )
                )
                for region in cell.bounding_regions:
                    # print(
                    #     "...content on page {} is within bounding box '{}'".format(
                    #         region.page_number,
                    #         format_polygon(region.polygon),
                    #     )
                    # )
                    contents.append(
                        "...content on page {} is within bounding box '{}'".format(
                            region.page_number,
                            format_polygon(region.polygon),
                        )
                    )

    # print("----------------------------------------")

    return result


In [28]:
import json

result = analyze_layout()
print(result)

# with open('/content/drive/MyDrive/github_repositories/chat-with-PDFs/data/土木技術管理規定集/test.json', 'w') as f:
#     json.dump(result, f, indent=2)

AnalyzeResult(api_version=2023-07-31, model_id=prebuilt-layout, content=UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549
FORM 10-Q
X :selected: QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the Quarterly Period Ended March 31, 2020
OR :unselected: TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the Transition Period From
to
Commission File Number 001-37845
MICROSOFT CORPORATION
WASHINGTON (STATE OF INCORPORATION)
91-1144442
(I.R.S. ID)
ONE MICROSOFT WAY, REDMOND, WASHINGTON 98052-6399 (425) 882-8080 www.microsoft.com/investor
Securities registered pursuant to Section 12(b) of the Act:
Title of each class
Trading Symbol
Name of exchange on which registered
Common stock, $0.00000625 par value per share
MSFT
NASDAQ
2.125% Notes due 2021
MSFT
NASDAQ
3.125% Notes due 2028
MSFT
NASDAQ
2.625% Notes due 2033
MSFT
NASDAQ
Securities registered pursuant to Section 12(g) of the Act:
NO